### Hyun-Joon Yang
### yanghyun@usc.edu
### QBIO 401
### Assignment 2

Four FASTQ files are placed on Blackboard:

<blockquote>
XI1_ATCACG_L001_R1_001.fastq <br>
XI1_ATCACG_L001_R2_001.fastq <br>
RETT-1_S1_L001_R1_001.fastq <br>
RETT-1_S1_L001_R2_001.fastq <br>
</blockquote>

These are next generation sequencing files from a paired-end run. The XI1 reads are mostly 250 base
pairs long, the RETT reads are mostly 150 base pairs long. Complicating the assignment, there are a few
reads that are a few bases shorter or longer than 250 or 150.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def readFASTQ(line_format, filename):
    """
    extract certain format line of each read
    :param line_format: number indicating which line to read
        (1 -> header, 2 -> sequence, 3 -> +, 4 -> quality score)
    :param filename: name of FASTQ file
    :return: list of extracted strings
    """
    result = []
    idx = 1
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if (idx - line_format) % 4 == 0 and line != '':
                idx = line_format
                result.append(line)
            idx += 1
    return result

def calcQScore(qsstr):
    """
    converts quality score string into list of actual scores
    :param qsstr: string of characters representing quality score
    :return: list of int representing quality score
    """
    qscores = []
    for qs in qsstr:
        qscores.append(ord(qs) - ord('!'))
    return qscores

#### 1. Write a Python function that counts the number of reads with the same, shorter and longer length than expected. For each of the four files print the output. Compare the results and describe your finding

In [ ]:
def countReads(filename, length):
    result = {'same': 0, 'shorter': 0, 'longer': 0}
    seqlist = readFASTQ(2, filename)
    for seq in seqlist:
        if len(seq) < length:
            result['shorter'] += 1
        elif len(seq) > length:
            result['longer'] += 1
        else:
            result['same'] += 1
    return result

In [ ]:
print(countReads('XI1_ATCACG_L001_R1_001.fastq', 250))
print(countReads('XI1_ATCACG_L001_R2_001.fastq', 250))
print(countReads('RETT-1_S1_L001_R1_001.fastq', 150))
print(countReads('RETT-1_S1_L001_R2_001.fastq', 150))

It seems that the XI1 files are very consistent with the lengths, with all 10,000 reads being of the same length. On the other hand, RETT files are less consistent, with about 1,000 reads being shorter and about 7,000 reads being longer than the expected length.

#### 2. Write a Python function that for each of the positions 1 to 250 (or 1 to 150) will compute the fraction of reads in the file with quality scores greater than or equal to 30 at that position. For each of the four files plot the output. Compare the results and describe your finding

In [ ]:
def calcQScoreFraction(filename):
    # get all the quality scores of reads
    qscores = readFASTQ(4, filename)
    # convert all qs strings to int & find max length
    maxlen = len(qscores[0])
    for i in range(len(qscores)):
        qscores[i] = calcQScore(qscores[i])
        if len(qscores[i]) > maxlen:
            maxlen = len(qscores[i])
    # make all reads the same size
    # input -1 for nonexistent positions
    for i in range(len(qscores)):
        if len(qscores[i]) != maxlen:
            for j in range(maxlen - len(qscores[i])):
                qscores[i].append(-1)
    # turn into numpy array
    X = np.array(qscores)
    # calculate fraction of reads >= 30 in qs
    # ignore positions with value -1
    numerator = np.sum(X >= 30, axis=0)
    denominator = np.sum(X != -1, axis=0)
    
    return maxlen, numerator / denominator

In [ ]:
maxlen_x1, result_x1 = calcQScoreFraction('XI1_ATCACG_L001_R1_001.fastq')
maxlen_x2, result_x2 = calcQScoreFraction('XI1_ATCACG_L001_R2_001.fastq')
maxlen_r1, result_r1 = calcQScoreFraction('RETT-1_S1_L001_R1_001.fastq')
maxlen_r2, result_r2 = calcQScoreFraction('RETT-1_S1_L001_R2_001.fastq')

In [ ]:
plt.figure()
plt.plot([i for i in range(maxlen_x1)], result_x1, 'ro')
plt.title('XI1_ATCACG_L001_R1_001.fastq')
plt.xlabel('Position')
plt.ylabel('Fraction of Reads with QS >= 30')
plt.figure()
plt.plot([i for i in range(maxlen_x2)], result_x2, 'rx')
plt.title('XI1_ATCACG_L001_R2_001.fastq')
plt.xlabel('Position')
plt.ylabel('Fraction of Reads with QS >= 30')
plt.figure()
plt.plot([i for i in range(maxlen_r1)], result_r1, 'bo')
plt.title('RETT-1_S1_L001_R1_001.fastq')
plt.xlabel('Position')
plt.ylabel('Fraction of Reads with QS >= 30')
plt.figure()
plt.plot([i for i in range(maxlen_r2)], result_r2, 'bx')
plt.title('RETT-1_S1_L001_R2_001.fastq')
plt.xlabel('Position')
plt.ylabel('Fraction of Reads with QS >= 30')

It seems that the XI1 reads decrease significantly in quality towards the end (starting at around 200th position). On the other hand, RETT reads remain relatively consistent in quality score, except at the beginning (around 15~40th positions) which sees  a drop in quality.

#### 3. Write another function that for k from 0 to 250 (or from 0 to 150) will count the number of reads in the file with exactly k positions with quality scores greater than or equal to 30. For each of the four files plot the output

In [ ]:
def processKQScore(filename):
    """
    goes through each read & stores number of qs >= 30 in a dictionary
    :param filename: name of file
    :return: dictionary that maps k (number of positions w qs >= 30) to number of reads
    """
    qsDict = {}
    # get all the quality scores of reads
    qscores = readFASTQ(4, filename)
    # look through all reads
    for qsstr in qscores:
        k = 0
        # look through each score of read & count number of scores >= 30
        qs = calcQScore(qsstr)
        for score in qs:
            if score >= 30:
                k += 1
        # if already in dictionary increment
        # if not, add key & set value to 1
        if k in qsDict:
            qsDict[k] += 1
        else:
            qsDict[k] = 1
    
    return qsDict

def countKQScore(qsDict, k):
    if k in qsDict:
        return qsDict[k]
    else:
        return 0

In [ ]:
qsdict_r1 = processKQScore('RETT-1_S1_L001_R1_001.fastq')
qsdict_r2 = processKQScore('RETT-1_S1_L001_R2_001.fastq')
qsdict_x1 = processKQScore('XI1_ATCACG_L001_R1_001.fastq')
qsdict_x2 = processKQScore('XI1_ATCACG_L001_R2_001.fastq')

In [ ]:
result_r1 = [countKQScore(qsdict_r1, k) for k in range(151+1)]
result_r2 = [countKQScore(qsdict_r2, k) for k in range(151+1)]
result_x1 = [countKQScore(qsdict_x1, k) for k in range(250+1)]
result_x2 = [countKQScore(qsdict_x2, k) for k in range(250+1)]

In [ ]:
plt.figure()
plt.plot([i for i in range(151+1)], result_r1, 'bo')
plt.title('RETT-1_S1_L001_R1_001.fastq')
plt.xlabel('k')
plt.ylabel('Number of Reads')
print(sum(result_r1))
plt.figure()
plt.plot([i for i in range(151+1)], result_r2, 'bx')
plt.title('RETT-1_S1_L001_R2_001.fastq')
plt.xlabel('k')
plt.ylabel('Number of Reads')
print(sum(result_r2))
plt.figure()
plt.plot([i for i in range(250+1)], result_x1, 'ro')
plt.title('XI1_ATCACG_L001_R1_001.fastq')
plt.xlabel('k')
plt.ylabel('Number of Reads')
print(sum(result_x1))
plt.figure()
plt.plot([i for i in range(250+1)], result_x2, 'rx')
plt.title('XI1_ATCACG_L001_R2_001.fastq')
plt.xlabel('k')
plt.ylabel('Number of Reads')
print(sum(result_x2))

#### TESTS

In [ ]:
import unittest
import os

class TestReadFASTQ(unittest.TestCase):
    def tearDown(self):
        if os.path.exists('test.txt'):
            os.remove('test.txt')
        
    def test_emptyFileName(self):
        self.assertRaises(FileNotFoundError, readFASTQ, 1, '')
    
    def test_blanksAtEOF(self):
        with open('test.txt', 'w') as f:
            print('\n\n', file=f)
        
        result = readFASTQ(1, 'test.txt')
        
        self.assertEqual([], result)
    
    def test_default(self):
        with open('test.txt', 'w') as f:
            for i in range(5):
                print('this is header', file=f)
                print('this is seq', file=f)
                print('this is addition', file=f)
                print('this is qs', file=f)
    
        result = readFASTQ(2, 'test.txt')
        
        expected = []
        for i in range(5):
            expected.append('this is seq')
        self.assertEqual(expected, result)
        
class TestCalcQScore(unittest.TestCase):
    def test_emptyString(self):
        scores = calcQScore('')
        
        self.assertEqual([], scores)
    
    def test_default(self):
        qsstr = '!;@J'
        
        scores = calcQScore(qsstr)
        
        expected = [0, 26, 31, 41]
        self.assertEqual(expected, scores)
        
class TestCountReads(unittest.TestCase):
    def tearDown(self):
        if os.path.exists('test.txt'):
            os.remove('test.txt')
            
    def test_default(self):
        seq = ['aaa', 'aaa', 'aaaa', 'aa', 'aaa']
        with open('test.txt', 'w') as f:
            for s in seq:
                print('this is header', file=f)
                print(s, file=f)
                print('this is addition', file=f)
                print('this is qs', file=f)
        
        result = countReads('test.txt', 3)
        
        self.assertEqual({'same': 3, 'shorter': 1, 'longer': 1}, result)

class TestCalcQScoreFraction(unittest.TestCase):
    def test_sameShape(self):
        pass
    
    def test_numResultEqualsMaxLen(self):
        pass
        
class TestCountKQScore(unittest.TestCase):
    def tearDown(self):
        if os.path.exists('test.txt'):
            os.remove('test.txt')
        
    def test_default(self):
        pass

unittest.main(argv=[''], verbosity=2, exit=False)